<a href="https://colab.research.google.com/github/Jakeh33/Computer-Vision/blob/main/Computer_Vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 6476 Computer Vision
- [课程主页](https://www.cc.gatech.edu/~hays/compvision/)
- [课程视频](https://omscs.gatech.edu/cs-6476-computer-vision-course-videos)
- [参考教材](http://szeliski.org/Book/)


# Introduction
- 计算机视觉：通过某些方式使得计算机能够对图/视频中的场景进行分析，给出场景中有什么，正在发生什么
- 人会被错觉误导：大脑通过图像的变化构建运动，对阴影的移动进行更改就会导致大脑构建的运动出现区别
- 视觉与图像处理的区别在于**构建对图像的描述**
- 课程构成：计算模型（数学原理）+算法+实践


# 图像组成和处理

## 将图像视为函数


- 将灰度图像视为二元函数$I(x,y)$
- 函数值从$[0,1]$，8位色彩使得上限可以是255
- RGB模型
$$f(x,y) = \begin{bmatrix} r(x,y)\\ g(x,y)\\ b(x,y) \end{bmatrix}$$
- 图像坐标系：左上角为原点，横轴x，纵轴y
- pixel = picture element
- 横坐标x = 列j = 宽度w
- 纵坐标y = 行i = 高度h
- 在matlab中，像素点的数据类型为uint8，即限幅[0,255]，因此对于图像叠加，先除再求和更准确
- 典型噪声
  - 椒盐噪声
  - 脉冲噪声
  - 高斯噪声
  - 噪声也可以视为函数，叠加在原始图像函数上得到新图像
- 噪声函数：`noise = randn(size(img)).*sigma`


## 滤波


### 基本概念

- 消除噪声：平滑操作，当前像素的值由周围像素的值决定
- 平均值假设：
  - 相邻像素的真值相近
  - 每个像素点的噪声相互独立
- 权值平均：基于值相近，离目标像素点越近的点权值越高
- 卷积核$(2k+1,2k+1)$
  - 均值滤波（盒子滤波器）
  $$G[i,j] = \frac 1 {(2k+1)^2} \sum^k_{u=-k} \sum^k_{v=-k} F[i+u,j+v]$$
  - 非均匀权重
  $$G[i,j] = \sum^k_{u=-k} \sum^k_{v=-k} H[u,v] F[i+u,j+v]$$
  称为互相关(cross-correlation) 记作$G =H \otimes F$
- 高斯滤波器
  - 距离目标位置越近影响越大
  - 举例：
  $${\frac 1{16}}\begin{bmatrix}1&2&1\\ 2&4&2\\ 1&2&1\end{bmatrix} $$
  $$h(u,v) = {\frac 1 {2\pi\sigma^2}}e^{-\frac{u^2+v^2}{2\sigma^2}}$$
  - 方差$\sigma^2$或者标准差$\sigma$：决定了平滑程度，越大越平滑
  - kernel 或者 mask的大小:`3*3,5*5,10*10,..`

- 中值滤波器
  - 当噪声是比较独立的噪点（比如椒盐噪声）时，图像像素值突变较多时
  - 将中心点的值用周围点的中值代替

- matlab 高斯滤波
  ```
  hsize = 31;
  sigma = 5;
  h = hspecial('gaussian',hsize,sigma)'
  % surf(h); 三维显示
  % imagesc(h); 二维显示
  outim = imfilter(im,h);
  ```

### 线性和卷积

- 线性：可加性$H(f1+f2) = H(f1)+H(f2)$、齐次性$(H(a·f1) = a·H(f1)$
- 脉冲：离散时域下就是单点，连续时域下是单位面积的区域信号
- 对单个像素值为1的施加$$\begin{bmatrix}a&b&c\\ d&e&f\\ g&h&i\end{bmatrix} $$
的滤波核，得到的脉冲响应为$$\begin{bmatrix}i&h&g\\ f&e&d\\ c&b&a\end{bmatrix}$$
- 互相关得到的结果是翻转的，在此基础上得到卷积的计算公式
$$G[i,j] = \sum^k_{u=-k} \sum^k_{v=-k} H[u,v] F[i-u,j-v]$$
记作$G=H\star F$
- 当kernel关于x和y对称时，互相关与卷积结果一致
- 在进行卷积操作时，先对H进行翻转$H^* \to _*H$,然后对应元素相乘相加
- 卷积的特性：线性、平移不变性（相同输入的结果不随位置和时间的改变而改变）、交换律、结合律、可微性
- 计算量大$N \times N$像素$W \times W$卷积核,需要计算$N^2 W^2$次，将H用行向量与列向量乘积表示，$G=H\star F =(C\star R)\star F = C\star (R\star F)$，只需计算$2WN^2$次
- 边界问题：如何对处在边界的像素求卷积
  - 取决于输出的尺寸[介绍](https://blog.csdn.net/leviopku/article/details/80327478)
    - full 全尺寸：从filter和image刚相交开始做卷积
    - same 与输入相同:当filter的中心(K)与image的边角重合时，开始做卷积
    - vaild 有效：当filter全部在image里面的时候，进行卷积
  - 边缘填充的方式：
    - 全黑或者全白`imfilter(f,g,0)`
    - 用傅里叶填充对边的边缘(构成周期循环图片堆）`imfilter(f,g,'circular')`
    - 边缘复制（将边缘值复制扩展）`imfilter(f,g,'replicate')`
    - 翻转复制边缘（将边缘带反向复制）`imfilter(f,g,'symmetric')`
- 模糊
  - 盒子滤波器、高斯滤波器...
  $${\frac 1{9}}\begin{bmatrix}1&1&1\\ 1&1&1\\ 1&1&1\end{bmatrix} $$
- 锐化
  - 核：值为2的脉冲滤波 - 盒子滤波器
 $$\begin{bmatrix}0&0&0\\ 0&2&0\\ 0&0&0\end{bmatrix} - {\frac 1{9}}\begin{bmatrix}1&1&1\\ 1&1&1\\ 1&1&1\end{bmatrix} $$ 减号后的部分为unsharp mask


### 模板式滤波器

- 归一化(normalize correlation)：对比不同滤波器在图像上的效果差别
  - 对滤波器进行归一化：使得标准差为1
  - 对与滤波核相乘的每个图像块进行归一化：使得图像块的标准差为1
  - NCC（normalized cross-correlation）：归一化后进行交叉互相关
- 与滤波核匹配度越高的部分得到的结果值越大
  - 直观理解：
    - 假设信号是标准差为1，中位为0的随机信号。即同时存在正值和负值信号。
    - 假设滤波核标准差为1，同时存在正值和负值
    - 对每个信号片段进行交叉互相关（对应位置相乘并相加)，
则正值$\times$正值+负值$\times$负值能够得到更大的结果
  - 因此滤波器能够有效保留图像中与滤波核结构一致的部分
- matlab:`c = normxcorr2(filter,img);figure surf(c),shading flat;`
- 二维图像模板匹配，找到匹配结果的左上角坐标
```
function[yIndex,xIndex] = find_template_2D(template,img)
  c = normxcorr2(template,img);
  [yRaw,xRaw] = find(c == max(c(:)));
  yIndex = yRaw - size(template,1)+1;
  xIndex = xRaw - size(template,2)+1;
end
```
- 不一定需要完全一致的模板，重要的是有着相同的结构，但是对于图中相似结构较多时，需要一个精确的模板才能找到最匹配的结果


## 边缘检测

### 一维 梯度

- 边缘的特点
  - 表面法向不连续性
  - 深度不连续性
  - 表面颜色不连续性
  - 光照不连续性（影子）
- 边缘在亮度图中表现类似悬崖（或者说阶跃），一阶导存在局部极值
- 涉及问题：挑选多大的邻域做检测以及如何检测
- 将不同的微分算子作为核计算图像梯度，设置不同的阈值提取边缘
  - 图像的梯度
  $$\nabla f = [\frac{\partial f}{\partial x},\frac{\partial f}{\partial y}]$$
  - 梯度的方向为像素值变化最大的方向，梯度的模决定了变化的程度
  $$ \theta = tan^{-1}({\frac{\partial f}{\partial y}/\frac{\partial f}{\partial x}}) \\ ||\nabla f|| = \sqrt{(\frac{\partial f}{\partial x})^2+
  (\frac{\partial f}{\partial y})^2} $$
- 由于图像为离散数据点，根据离散偏微分
$\frac{\partial f(x,y)}{\partial x} \approx \frac{f(x+1,y) -f(x,y)} 1$
  - 挑选竖直边缘，$ker = [-1,1]$
  - 挑选水平边缘，$ker = [-1,1]^T$
- 根据上述分析，建立
$$H=\begin{bmatrix}0&0\\ -1&1\\ 0&0 \end{bmatrix}$$
但是这个核不够理想，因为它不存在中间点，而且只计算了一个方向，因此改进得到
$$H=\begin{bmatrix}0&0&0\\ -0.5&0&0.5\\ 0&0&0 \end{bmatrix}$$
- Sobel 算子
  - x方向
  $$\frac{1}{8}\begin{bmatrix}-1&0&1\\ -2&0&2\\ -1&0&1 \end{bmatrix}$$
  - y方向
  $$\frac{1}{8}\begin{bmatrix}-1&-2&-1\\ 0&0&0\\ 1&2&1 \end{bmatrix}$$
  - `filt = fspecial('sobel'); outim = imfilter(double(im),filt);`(默认使用correlation)

```
  [gx gy] = imgradientxy(img,'');%默认使用Sobel,但是未做标准化(即没除8）
  imshow((gx+4)/8); %将结果调整到[0,1]，or imshow(gx,[-4,4]);
  [gmag gdir] = imgradient(gx,gy); %得到梯度
  imshow(gmag/ (4*sqrt(2))));
```
- 噪声的存在对边缘检测影响很大，因此先平滑再检测，再根据卷积原理：
$\frac{\partial}{\partial x}(h \star f) = \frac{\partial h}{\partial x} \star f$，寻找最值的过程又进行了一次偏导，$\frac{\partial^2 h}{\partial x^2} \star f$。此时找到结果中的零点，就对应边缘
- 通过上述推导，为了减少计算量，对核做偏微分而非图像

### 二维

- 高斯滤波器微分
$$(I \otimes g) \otimes h_x = I \otimes (g \otimes h_x) $$
- 小$\sigma$能够检测细节特征，反之检测大尺度边缘
- 一般的边缘检测步骤：
  - 平滑减少噪声并计算梯度
  - 阈值分割保留部分梯度值
  - “Thin”保留局部边缘像素
  - 连接边缘像素（双阈值）
- Canny检测
  - 对图像使用微分高斯滤波器
  - 计算梯度
  - 非极大值抑制：将多像素宽度边缘带降为单像素宽度
  - `edge(image,'canny');`
  - 先用高阈值找到强边缘，再用低阈值找到弱边缘，(遍历强边缘，将八邻域内高于低阈值的点标记为边）将强边缘沿着弱边缘延伸
- Non-maximal suppression（非极大值抑制）
  - 沿着梯度的方向找到局部极大值
  - 由于像素点是离散的，当梯度方向上没有像素点时，使用相邻像素点进行线性插值，得到近似值 
- 对高斯滤波做二阶微分
  - 有两个方向可以选择
  - 应该使用高斯-拉普拉斯形式
$$\nabla^2 h = \frac{\partial^2 f}{\partial x^2} +\frac{\partial^2 f}{\partial y^2} $$
  - 零点为边缘

```
img = rgb2gray(origin_img);
h = fspecial('gussian',[11,11],4);
smooth_res = imfilter(img,h);
canny_edge = edge(img,'canny');
% 平滑后的图像会丢失很多细节
log_edge = edge(img,'log');%拉普拉斯高斯； canny的结果更连续和平滑
```

## 霍夫变换(Hough Transform)
- 找到可以参数化的特殊形状
- 需要当前检测位置的周边信息支持
- 计算复杂度需要尽可能小
- 优点
  - 每个点独立投票，可以应付遮挡
  - 对噪声有一定的鲁棒性
  - 可以在图像中找到多个个体
- 缺点
  - 算法复杂度随参数个数指数增长
  - 对非典型形状效果不太行
  - 比较难挑选出一个合适的网格大小

### 直线

- 边缘检测会带来更多的无关信息，会有部分直线未被检测，存在噪点
- 单纯根据离散的边缘点可以排列组合出多条不同的直线
- 计算量的限制，不可能去检测每一条可能的直线，因此提出了Voting的概念
- Voting
  - 让特征给满足条件的模型投票
  - 循环特征（边缘点），决定是否满足模型参数
  - 找出得票多的模型参数
  - 噪声和散乱特征也会投票，但是这些离散投票不会影响到好特征的投票结果
- 霍夫变换
  - 每一个边缘点都将给可行直线投票
  - 找到得票多的直线
- 霍夫空间
  - 在图像平面里，直线的表示为$y = m_0x+b_0$
  - 变换到一个在霍夫平面（横轴m，纵轴b）中的点$(m_0,b_0)$
  - **图像中的一条线对应霍夫平面的一个点**
  - 像平面的点$(x_0,y_0)$，经过它的直线满足$y_0=mx_0+b$
  - 对应霍夫平面的一条线$b=-x_0m+y_0$
  - **图像中的一个点对应霍夫平面的一条线**
  - 因此图像中两点对应的霍夫平面直线的交点$(m,b)$，对应两点连线的直线方程
- 直线表示法：斜距法在面对竖直线时给出的结果不适合计算，因此选用类极坐标系表示直线$(d,\theta)$，对应原点到直线的垂线的长度以及该垂线与x轴的夹角
  - $x\cos \theta +y \sin \theta = d, d\geq 0 $ 
  - 或者d可以为负数，$x\cos \theta - y \sin \theta = d$ 
- 算法思想
  - 将每个边缘点都在霍夫空间中给出一条直线
  - 将空间分成不同的网格，经过直线最多的网格对应直线的参数‘’
- 算法实现
  - 初始化$H[d,\theta] = 0$(矩阵，具体大小取决于区间和分段间距）
  - 对所有$E(x,y)$中的边缘点，
    - 对所有$\theta$ 从$0 \to 180$,$d = x\cos \theta - y \sin \theta$，区间对应块的$H[d,\theta] += 1 $
    - 找到使得$H[d,\theta]$最大时的$(d,\theta)$
    - 检测得到的直线在图中表示为$d = x\cos \theta - y \sin \theta$
  - 空间复杂度：k个分块，n个参数$k^n$
  - 时间复杂度：取决于特征点的个数
- 用梯度的角度代替遍历整个$\theta$定义域，能够有效减少计算量

```
grays = rgb2gray(img);
edges = egde(grays,'canny');
% 霍夫变换 横轴角度，纵轴距离，绘制投票结果
[accum,theta,rho] = hough(edges);
figure,imagesc(accum,'XData',theta,'YData',rho);
% 找到投票结果峰点并在上图中用红框标出
peaks = houghpeaks(accum,100);
hold on; 
plot(theta(peaks(:,2)),rho(peaks(:,1)),'rs');
% 寻找直线并绘制
line_segs = houghlines(edges,theta,rho,peaks);  % [point1，point2,theta,pho]
figure,imshow(img)
hold on;
for k = 1:length(line_segs)
  endpoints = [line_seg(k).point1;line_seg(k).point2];
  plot(endpoints(:,1),endpoints(:,2),'g-');
end
hold off;
% 改进参数(直线上边缘点个数的底线，默认0.5；局部极值计算区域（theta，rho）组成的区域)，挑选更合适的点
peaks = houghpeaks(accum,100,'Threshold',ceil(0.6*max(accum(:))),'NHoodSize',[5 5]);
line_segs = houghlines(edges,theta,rho,peaks,'FillGap',50,'MinLength',100);
```


### 圆

- 圆的表示法（圆心坐标+半径）：
  - $(x_i-a)^2+(y_i-b)^2 = r^2$
  - 假设半径r已知，在霍夫空间中只需表示$(a,b)$，图像空间中的点在霍夫空间（横轴a，纵轴b）中表示为圆
  - $(a-x_i)^2+(b-y_i)^2 = r^2$
  - 投票选出圆心点$(a,b)$

![霍夫变换-圆](https://img-blog.csdn.net/20180921110025367)

- 当半径未知时
  - 霍夫空间扩展成三维，增加半径r
  - 不同的r值在$(a,b)$平面上形成不同的圆，组合形成一个倒锥形的外表面
  - 此时三维的统计十分麻烦，因此为了简化问题，引入梯度值
  - 可以确定圆心必定在梯度方向上，因此，将备选简化为在霍夫空间中的直线
  - $a = x-r\cos(\theta);b = y+r\sin(\theta)$

![半径未知-圆](https://img-blog.csdn.net/20180921122106732)
- 霍夫梯度法：
  - 速度快
  - 精确度低一些
  
```
For edge in E(x,y):
  For r in R(possible):
    For theta in T(gradient):
      a = x - r*cos(theta)
      b = y + r*sin(theta)'
      H[a,b,r] += 1
    end
  end
end
```

### 广义霍夫变换（模板匹配）

![从特例开始](https://img-blog.csdnimg.cn/20200426213947497.JPG)

- 非解析模型（不规则形状）
  - 对模板元素建立R-Table(Hough表）
    - 任意选取一个参考点
$C(x_c,y_c)$
    - 任意边缘点的梯度角度$\phi$，以及边缘点与参考点之间的偏差向量（用角度$\alpha$和长度r表示）
    - 选取梯度角度$\phi$作为表格的索引，将$(\alpha,r)$作为对应索引的元素
    - 遍历边缘点，补全表格
- 特例（目标的大小和方向与模板一致）
  - 对目标图片进行边缘检测，遍历每个边缘点
  - 根据当前边缘点的梯度角度$\phi$，从Table中找到$\phi$对应的所有偏差向量，逐个应用到边缘点上，得到参考点的坐标并投票
  - 得到参考点之后，将模板图片应用到参考点
- 一般情况（改变大小和方向）
  - 引入旋转角度($\theta)$和缩放因子s
  - 找到$\phi - \theta$对应的偏移向量
  - 对每个偏移向量乘上缩放因子s同时旋转$\theta$
  - 遍历所有可能的角度和缩放因子
  - 投票次数在特例基础上$*\theta_{Dim}*s_{Dim}$
  - 得到旋转角度，缩放因子和参考点坐标

![一般情况](https://img-blog.csdnimg.cn/2020042623361164.JPG)

- **注意点**：
在应用旋转时使用的是逆时针旋转角度，而图像坐标系与传统的笛卡尔系之间又存在一个对称变换，图像坐标系下的逆时针旋转在笛卡尔系下变现为顺时针

- 优点：
  - 广义霍夫变换本质上是一种用于物体识别的方法。
  - 它对部分或轻微变形的形状鲁棒性好(即对遮挡下的识别鲁棒性好)。
  - 对于图像中存在其他结构(即其他线条，曲线等)干扰，鲁棒性好。
  - 抗噪声能力强。
  - 一次遍历即可找到多个同类目标。
- 缺点：
  - 它需要大量的存储和大量的计算(但是它本质上是可并行化的)。

- 更现代的应用
  - 将纹理作为visual codeword 
  - 基于目标纹理，给出当前纹理的偏移向量
- Training：
  - codeword获取：提取数据集图像的感兴趣点（POI），对感兴趣点周围图像采样得到特征，对特征结果进行聚类，聚类得到结果为codeword（全自动实现）
  - 感兴趣点：对POI周围的块进行匹配，从第一步得到的备选集中选出最接近的纹理信息，打上聚类的结果label
  - 偏移向量：将匹配后的纹理当做边缘点，应用广义霍夫变换，将label作为索引，得到偏移向量
- 在识别中的应用
  - 纹理信息为轮胎，R-Table记录轮胎每个有两个偏移向量
  - 交点对应参考点，组成一辆车

![example](https://img-blog.csdn.net/20180924174717874)

## 频域分析
- 从频域来分析图像

### 傅里叶变化

- 图像分解（使用Basis sets）
  - 基组（basis set）
    - 向量空间V的基B能够组成V的一个线性无关子集
    - 通常会选择正交向量
  - 作用
    - 基于线性假设，将图像分解为基向量的组合，外部操作可以视作对每个独立的基施加后的结果进行叠加
  - N*M图像，如果选择$[00...1...00]^T$作为独立基向量，满足条件但是不实用
  - 在不同方向上的变化速率作为基

![傅里叶基](https://img-blog.csdnimg.cn/20181104173926785.png)

- **核心思想：任意周期函数都可以写成不同频率的三角函数的组合**
- 傅里叶级数：
  - 基础组件 $A\sin(\omega x+\varphi)$
  - 方波信号可以表示为$A\sum_{k=1}^{\infty}\frac{1}{k} \sin(2\pi kt)$
  - 分析图像时，相位的影响不大，而在图像重建时，相位影响比较大
- 傅里叶变换
  - $f(x) \to F(\omega)$ 后者需要同时包含对应的幅值和相位，因此将其引入复数域
  - $F(\omega) = R(\omega) + iI(\omega); A = \pm \sqrt{R^2+I^2}; \varphi = \tan^{-1} \frac{I}{R}$
  - 其中实部为偶函数，虚部为奇函数
- 计算傅里叶变换
  - 两个不同频率的正弦函数乘积的无穷积分为0 $\int_{-\infty}^{\infty} \sin(ax+\phi) *\sin(bx+\varphi)dx = 0;\quad \text{if}\quad a \neq b$
  - 而在a=b时，积分结果为无穷
  - 假设$f(x) =\cos (2\pi \omega x) \quad C(u) = \int_{-\infty}^{\infty} f(x) \cos(2\pi ux)dx$
  - 当$u=\pm \omega$时，结果为无穷，在频谱上变现为两个脉冲信号  
  ![spectrum](https://img-blog.csdnimg.cn/20181102205634393.png)
- 推广到标准形式
  - 基础单元：将信号从空间x变换到频域u
  - $F(u) = \int_{-\infty}^{\infty} f(x)e^{-i2\pi ux}dx$
  - $e^{ik} = \cos k+ i\sin k $
  - 逆变换$f(x) = \int_{-\infty}^{\infty} F(u)e^{i2\pi ux}du$
- 局限性
  - 不同于傅里叶级数，考虑无穷周期信号，在傅里叶变换时，我们考虑有界函数
  - 即当原始$|f(x)|$可积时，FT存在
  - 考虑在$[-\frac{T}{2},-\frac{T}{2}]$宽度内的信号，引出了离散傅里叶变换，更适合计算机领域
- 离散FT
  - $F(k) = \frac{1}{N} \sum_{x=0}^{x=N-1}f(x)e^{-i \frac{2\pi kx}{N}}$
  - 信号x从起点到终点$[0,N-1]$，k对应整个信号周期中重复的次数
  - $k \in [-\frac{N}{2},\frac{N}{2}] 时等式有意义$
  - k的定义域意味着最快（最多在N个信号长度中重复N/2次）
  - 举例说明，在图像中，像素值[0,1,0,1,...,0,1]变化，N个像素，重复[0,1]片段N/2次
- 扩展到二维
  $$F(u,v) = \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} f(u,v)e^{-i 2\pi (ux+vy)}dx dy$$
  $$F(k_x,k_y) = \frac{1}{N} \sum_{x=0}^{x=N-1}\sum_{y=0}^{y=N-1}f(x,y)e^{-i \frac{2\pi (k_xx+k_yy)}{N}}$$
  - 对图像做傅里叶变换通常会得到一个中心点亮，向四周变暗的结果
  - 中心对应低频，外围对应高频，将高频区域置0，做逆变换得到原始图像滤去高频后的结果
  - 将中心区域置0，逆变换得到边缘图像，对应高频信号组成边缘信息
- 频谱
  - 绘制不同频率对应的信号的power(对应傅里叶基的幅值）  
  ![spectrum](https://img-blog.csdnimg.cn/2018110418324466.png)

### 频域中的卷积
- 空间域的卷积 等价于 频域的乘积
- 根据对称性 空间域的乘积 等价于 频域的卷积

- 假设$g= f \star h$
$$ G(u) = \int_{-\infty}^{\infty} g(x)e^{-i2\pi ux}dx \\ 
= \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} f(\tau)h(x-\tau)e^{-i2\pi ux}d\tau dx \\ 
= \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} [f(\tau) e^{-i2\pi u\tau}d\tau] [h(x-\tau)e^{-i2\pi u(x-\tau)}dx] \\
= \int_{-\infty}^{\infty} [f(\tau) e^{-i2\pi u\tau}d\tau] \int_{-\infty}^{\infty} [h(x')e^{-i2\pi ux'}dx'] \\
= F(u)H(u)
$$
- 通过使用傅里叶变化将计算量大的卷积转变为频域的乘积再进行逆变换得到卷积结果，减少计算量  
![gaussian](https://img-blog.csdnimg.cn/20181105165935800.png)
- 例子（平滑/模糊）
  - 对高斯核进行傅里叶变换,结果依然是高斯 
  $h(x) = \frac{1}{\sqrt{2\pi}\sigma}exp[-\frac{x^2}{2\sigma^2}']$  
  $H(u) = exp[-\frac{1}{2} (2\pi u)^2 \sigma^2]$
  - 空间“瘦”高斯核$\to $频域“胖”高斯核
  - 将变换后的卷积与原始信号相乘，即保留低频信号，减小高频信号
- 低通和高通滤波
  - 只保留低频信息-低通-振铃效应（ringing）
    - 将其反变换回空间域，相当于应用了一个sinc函数作为滤波器
  - 只保留高频信息-高通-类边缘
  
![property](https://img-blog.csdnimg.cn/2018110610240179.png)

- 尺度变换性质
  - $ax,a>1$,相当于压缩信号，使得相同区域内重复次数更多，高频分量增大，频带变宽

![常用](https://img-blog.csdnimg.cn/20181106111605232.png)



### 混淆（Alias）
- 应用频域的卷积（空间域的乘积）

- 脉冲系列在频域中也为脉冲序列(在空间域和频域的的间隔分别为为$x_0, \frac{1}{x_0}$
- 将连续信号采样为离散信号，离散信号重建为连续信号
- 一维分析（chirp信号——信号从低频平滑过渡到高频）
  - 麦克风录音原理：声波带动磁铁运动，使得电压连续变化，通过A/D转换，得到离散数字信号，再通过D/A转换恢复成原始声音 
  - 欠采样：采样间距过大，丢失细节信息，无法将低频信号与高频信号区分开
    - 混淆：信号“伪装”成其他频率的信号
    - 视频录制的螺旋桨时而正转时而逆转就是典型的欠采样结果  
    ![Alias实例](https://img-blog.csdnimg.cn/20181108180817338.png)
- 反混淆
  - 增加采样频率(但是不可能无限增加)
  - 抑制高频信号（虽然会损失一些信息，但是好过混淆）
  - 引入低通滤波器
- 数学分析
  - 定义comb函数（脉冲序列）：
  $comb_M[x]= \sum_{k=-\infty}^{\infty} \delta[x-kM]$,M是整数
  - $FT(comb_M(x)) = \frac{1}{M} comb_{\frac {1}{M}} (u)$
  - 扩展到二维
  $$FT(comb_{M,N}(x,y)) = \frac{1}{MN} comb_{\frac {1}{M}, \frac {1}{N}} (u)\\
  comb_{M,N}(x,y)) = \sum_{k=-\infty}^{\infty} \sum_{l=-\infty}^{\infty}\delta(x-kM,y-lN)$$

- 采样
  - 将信号与采样序列相乘得到采样结果
  $f(x)·comb_M(x)$
  - 即在频域做卷积
  $F(u)\star comb_{\frac{1}{M}}(u)$
  - 假设卷积结果在频域内可以划分为多个重复的有界频带，则可以通过此频带重建信号  
  ![低频](https://img-blog.csdnimg.cn/20181109182214194.png)
  - 对于高频，直接FT变换+卷积会导致混淆，因此需要先用反混淆滤波器滤去高频  
  ![高频](https://img-blog.csdnimg.cn/20181109192106993.png)
- 在降采样的过程中，如果直接采样，易出现混淆
- 可以先做高斯滤波，再进行降采样，结果会更接近原图的模糊化
- 人眼对频率的感知受到很多因素的影响（比如对比度）
![C-S 对比度敏感曲线](https://www.pnas.org/content/pnas/112/3/875/F1.medium.gif)

- 图像压缩
  - 离散余弦变换（Discrete Cosine Transform）:将原图分割成多块8*8的区域，当区域内像素值变化平滑时（需要高频余弦的组合来拟合），当像素值变化较大时（拟合组合中会出现低频）
  - 人眼对高频的敏感度不如低频
  - 核心思想是利用较少的不同数值来表示图像
  - 通过量化矩阵（低频-左上角区域数值小，高频-右下角区域数值大）表示压缩程度
